In [5]:
from Coletor import Coletor
from Indexador import Indexador
from collections import defaultdict


In [6]:
class Buscador:
    def __init__(self, indexador):
        self.indexador = indexador

    def processar_consulta(self, consulta):
        termos = consulta.lower().split()
        resultados = None
        for termo in termos:
            if resultados is None:
                resultados = set(self.indexador.buscar(termo))
            else:
                resultados &= set(self.indexador.buscar(termo))
        return list(resultados)

    def calcular_precisao_revocacao(self, resultados, relevantes):
        tp = len(set(resultados) & set(relevantes))
        precisao = tp / len(resultados) if resultados else 0
        revocacao = tp / len(relevantes) if relevantes else 0
        return precisao, revocacao

    def classificar_resultados(self, resultados, consulta):
        pesos = defaultdict(int)
        for termo in consulta.lower().split():
            pesos[termo] += 1
        resultados_ponderados = [(url, sum(self.indexador.indice_invertido[termo]['idf'] * pesos[termo] for termo in consulta.lower().split())) for url in resultados]
        resultados_ordenados = sorted(resultados_ponderados, key=lambda x: x[1], reverse=True)
        return [url for url, _ in resultados_ordenados]

In [7]:
urls = [
    'https://www.nasa.gov',
    'https://github.com'
    # Adicione mais URLs específicas conforme necessário
]

# Criar instância do Coletor com as novas URLs
coletor = Coletor(urls=urls)

# Proceder com a coleta e indexação
coletor.coletar()
indexador = Indexador(coletor)
indexador.indexar_documentos()


# Criar instância do Buscador
buscador = Buscador(indexador)

# Processar consulta
consulta = "machine learning"
resultados = buscador.processar_consulta(consulta)

# Exibir resultados
print("Resultados da busca:")
for resultado in resultados:
    print(resultado)